In [1]:
import numpy as np
import pandas as pd
import scipy.io as sio
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

In [3]:
# load data (.mat)
data_sa = sio.loadmat(r'D:\code\data\FSR-iFMG\FSR_up_stairs')['FSR_up_stairs']
data_sd = sio.loadmat(r'D:\code\data\FSR-iFMG\FSR_down_stairs')['FSR_down_stairs']
data_walk = sio.loadmat(r'D:\code\data\FSR-iFMG\FSR_walk')['FSR_walk']
# set window increase & window length
window_i = 20
window_l = 100 #samplerate100, 40 50 60 70 80 90 100

def get_feature(data):  # calculate features
    feature = np.zeros(((data.shape[1] - window_l)//window_i + 1, 25), dtype = float)
    for i in range(5):
        for j in range((data.shape[1] - window_l) // window_i):
            feature[j, 5 * i] = np.mean(data[i, window_i*j : window_i*j + window_l])
            feature[j, 5 * i + 1] = np.std(data[i, window_i*j : window_i*j + window_l])
            feature[j, 5 * i + 2] = np.var(data[i, window_i*j : window_i*j + window_l])
            feature[j, 5 * i + 3] = np.mean(abs(data[i, window_i*j : window_i*j + window_l] - np.mean(data[i, window_i*j : window_i*j + window_l])))
            feature[j, 5 * i + 4] = max(data[i, window_i*j : window_i*j + window_l]) - min(data[i, window_i*j : window_i*j + window_l])
    return feature

f_data = []
label = []

f_walk = get_feature(data_walk.T)
f_stairA = get_feature(data_sa.T)
f_stairD = get_feature(data_sd.T)

for i in range(f_walk.shape[0]):
    label.append(0)

for i in range(f_stairA.shape[0]):
    label.append(1)

for i in range(f_stairD.shape[0]):
    label.append(2)
    
f_data = np.concatenate((f_walk, f_stairA, f_stairD), axis = 0)
print(f_data.shape)
print(len(label))



(2845, 25)
2845


In [4]:
print(data_sa.shape[1])

5


In [5]:
# randomly decvide data set
train_data_r, test_data_r, train_label, test_label = train_test_split(f_data, label, test_size = 0.2, random_state = None, shuffle = False)
scaler = preprocessing.StandardScaler().fit(train_data_r)
train_data = scaler.transform(train_data_r)
test_data = scaler.transform(test_data_r)
'''
c = svm.SVC(C = 52, kernel = 'rbf', gamma = 0.1, decision_function_shape = 'ovr')
c.fit(train_data, train_label)
print("train acc:", c.score(train_data, train_label))
print('test acc:', c.score(test_data, test_label))
test_pred = c.predict(test_data)
print(confusion_matrix(test_label, test_pred))
'''
parameters = {'gamma':[0.0001, 0.001, 0.01, 0.1, 1, 10], 'C':np.linspace(1, 100, 100)}
gs = GridSearchCV(svm.SVC(), parameters, refit = True, cv = 5, verbose = 1, n_jobs = -1)
gs.fit(train_data, train_label)
print('best parameter: ', gs.best_params_)
print('best efficience:', gs.best_score_)
print('report: ', classification_report(test_label, gs.predict(test_data_r)))

Fitting 5 folds for each of 600 candidates, totalling 3000 fits
best parameter:  {'C': 4.0, 'gamma': 0.1}
best efficience: 0.9956072874493926
report:                precision    recall  f1-score   support

           0       0.00      0.00      0.00       265
           1       0.26      1.00      0.42       149
           2       0.00      0.00      0.00       155

    accuracy                           0.26       569
   macro avg       0.09      0.33      0.14       569
weighted avg       0.07      0.26      0.11       569



D:\anaconda\installation\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\installation\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
D:\anaconda\installation\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [6]:
c = svm.SVC(C = 6, kernel = 'rbf', gamma = 0.1, decision_function_shape = 'ovr')# 
c.fit(train_data, train_label)
print("train acc:", c.score(train_data, train_label))
print('test acc:', c.score(test_data, test_label))
test_pred = c.predict(test_data)
print(confusion_matrix(test_label, test_pred))

train acc: 0.9991212653778558
test acc: 0.9964850615114236
[[265   0   0]
 [  0 148   1]
 [  0   1 154]]
